In [1]:
# Import dependencies

import numpy as np
import pandas as pd
import requests
import json
import time

from scipy.stats import linregress
from matplotlib import pyplot as plt

from citipy import citipy

In [2]:
# Import OpenWeather API key

from config import api_key

In [3]:
# Generate a random list of cities with citipy

# Generate a random list of coordinates
# "The valid range of latitude in degrees is -90 and +90 for the southern and northern hemisphere respectively. Longitude is in the range -180 and +180" https://stackoverflow.com/questions/15965166/

latitude = []
longitude = []

for lat in np.random.randint(-90,90,500):
    latitude.append(lat)

for long in np.random.randint(-180,180,500):
    longitude.append(long)

lat_long = []
for i in range(len(latitude)):
    lat_long.append((latitude[i], longitude[i]))

# lat_long = tuple(zip(latitude,longitude))

# lat_long

# Locate nearest cities to coordinates
cities = []

for lat,long in lat_long:
    city = citipy.nearest_city(lat,long)
    city_name = city.city_name
    cities.append(city_name)

cities

['saint-joseph',
 'cape town',
 'avarua',
 'cape town',
 'hobart',
 'nikolskoye',
 'christchurch',
 'palmer',
 'vaini',
 'high rock',
 'praia da vitoria',
 'mamallapuram',
 'port hardy',
 'georgetown',
 'piacabucu',
 'nyamuswa',
 'kuala kedah',
 'yellowknife',
 'victoria',
 'tasiilaq',
 'punta arenas',
 'qaanaaq',
 'nhulunbuy',
 'hermanus',
 'hongjiang',
 'avarua',
 'ponta do sol',
 'barentsburg',
 'tarata',
 'yellowknife',
 'kingussie',
 'derzhavinsk',
 'savinka',
 'mataura',
 'puerto carreno',
 'mahebourg',
 'castro',
 'kushmurun',
 'chauk',
 'faanui',
 'mataura',
 'ilulissat',
 'vostok',
 'esperance',
 'butaritari',
 'punta arenas',
 'wazzan',
 'rikitea',
 'marcona',
 'iqaluit',
 'busselton',
 'mar del plata',
 'mar del plata',
 'axim',
 'callaway',
 'vaini',
 'saldanha',
 'nemuro',
 'guerrero negro',
 'butaritari',
 'arraial do cabo',
 'cabo san lucas',
 'rikitea',
 'busselton',
 'atuona',
 'lucea',
 'hithadhoo',
 'kapaa',
 'palora',
 'abbeville',
 'hakkari',
 'hope mills',
 'valle

In [35]:
# Testing data retrieval

base_url = "http://api.openweathermap.org/data/2.5/weather?".format(target_city,api_key)
target_url = base_url + "q=" + city_name + "&appid=" + api_key

city_name = "albany"

print("Tesing OpenWeatherMap API")

response = requests.get(target_url)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)
data_test = response.json()
# print(json.dumps(data_test, indent=4, sort_keys=True))

main = data_test["main"]
temperature = main["temp"] # Fetch temperature
humidity = main["humidity"] # Fetch humidity

wind = data_test["wind"]
wind_speed = wind["speed"] # Fetch wind speed

clouds = data_test["clouds"]
cloudiness = clouds["all"] # Fetch cloudiness


print(f"{city_name.title():-^30}")
print(f"Temperature: {temperature}")
print(f"Humidity: {humidity}")
print(f"Wind speed: {wind_speed}")
print(f"Cloudiness: {cloudiness}")



Tesing OpenWeatherMap API
------------Albany------------
Temperature: 274.48
Humidity: 69
Wind speed: 2.24
Cloudiness: 96


In [ ]:
# Data retrieval

